In [1]:
import os
from glob import glob
import numpy as np
import dask
import xarray as xr
import xrft
import cartopy.crs as ccrs
from cmocean import cm
import h5py

from matplotlib import pyplot as plt
%matplotlib inline

from mitequinox.utils import *

/home1/datahome/xyu/.miniconda3/envs/equinox/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home1/datahome/xyu/.miniconda3/envs/equinox/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home1/datahome/xyu/.miniconda3/envs/equinox/lib/python3.6/site-packages/cmocean/tools.py:76: MatplotlibDeprecationWarning: The is_string_like function was deprecated in version 2.1.
  if not mpl.cbook.is_string_like(rgbin[0]):
/home1/datahome/xyu/.miniconda3/envs/equinox/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [2]:
from dask_jobqueue import PBSCluster
local_dir = os.getenv('TMPDIR')
cluster = PBSCluster(local_directory=local_dir)
#print(cluster.job_script())
w = cluster.start_workers(28)

/home1/datahome/xyu/.miniconda3/envs/equinox/lib/python3.6/site-packages/distributed/utils.py:128: RuntimeWarning: Couldn't detect a suitable IP address for reaching '8.8.8.8', defaulting to '127.0.0.1': [Errno 101] Network is unreachable
  % (host, default, e), RuntimeWarning)


In [3]:
# get dask handles and check dask server status
from dask.distributed import Client
client = Client(cluster)

In [20]:
client

Client Scheduler: tcp://10.148.1.20:57096 Dashboard: http://10.148.1.20:8787/status,Cluster Workers: 34 Cores: 136 Memory: 1700.00 GB


---


# 1. Drifter data files

In [5]:
# The prcessed dataset includes both the GPS- and Argo-tracked surface drifters.
# 2-month blocks with overlapping by 50%

data_dir = '/home1/datawork/xyu/Drifters/'

drifter_dataset = h5py.File(data_dir+'Drifter_dataset_processed.mat','r') 
list(drifter_dataset.items())

# mlat2: mean latitude over 60 days,     126303*1
# mlon2: mean longitude over 60 days,    126303*1

# qd: drogued drifters,                  126303*1
# qmed200: median water depth > 200 m,   126303*1
# qu: undrogued drifters,                126303*1

# uv2: (u, v),                           126303*1440

[('mlat2', <HDF5 dataset "mlat2": shape (126303, 1), type "<f8">),
 ('mlon2', <HDF5 dataset "mlon2": shape (126303, 1), type "<f8">),
 ('qd', <HDF5 dataset "qd": shape (126303, 1), type "|u1">),
 ('qmed200', <HDF5 dataset "qmed200": shape (126303, 1), type "|u1">),
 ('qu', <HDF5 dataset "qu": shape (126303, 1), type "|u1">),
 ('u', <HDF5 dataset "u": shape (126303, 1), type "<f8">),
 ('uv2', <HDF5 dataset "uv2": shape (126303, 1440), type "|V16">),
 ('v', <HDF5 dataset "v": shape (126303, 1), type "<f8">)]

In [6]:
# turn mean-lat into xarray_DataArray
lat2 = drifter_dataset['mlat2']
lat2 = np.array(lat2)
lat = xr.DataArray(lat2, dims=['lat', 'none'])
lat = xr.DataArray.squeeze(lat, dim='none')
lat = np.real(lat).rename('lat')
lat

<xarray.DataArray 'lat' (lat: 126303)>
array([ 39.51519 ,  39.682915,  40.437454, ..., -60.776903,  32.794351,
        27.131247])
Dimensions without coordinates: lat

In [7]:
# turn mean-lat into xarray_DataArray
lon2 = drifter_dataset['mlon2']
lon2 = np.array(lon2)
lon = xr.DataArray(lon2, dims=['lon', 'none'])
lon = xr.DataArray.squeeze(lon, dim='none')
lon = np.real(lon).rename('lon')
lon

<xarray.DataArray 'lon' (lon: 126303)>
array([150.087397, 151.958754, 152.837519, ..., 224.863612, 284.83327 ,
       285.592734])
Dimensions without coordinates: lon

In [8]:
# turn drogued_information into xarray_DataArray
qd = drifter_dataset['qd']
qd = np.array(qd)
qd = xr.DataArray(qd, dims=['qd', 'none'])
qd = xr.DataArray.squeeze(qd, dim='none')
qd = np.real(qd).rename('qd')
qd

<xarray.DataArray 'qd' (qd: 126303)>
array([0, 0, 0, ..., 1, 1, 1], dtype=uint8)
Dimensions without coordinates: qd

In [9]:
# (u,v) --> u+iv
uv = drifter_dataset['uv2'].value.view(np.double).reshape((126303, 1440, 2))
uv_drifters = uv[:,:,0] + 1j*uv[:,:,1]
uv_drifters.shape  # result is (10, 10, 10)
uv_drifters

array([[-0.2105+0.2479j, -0.1988+0.2534j, -0.1829+0.2546j, ...,
        -0.1973-0.0078j, -0.1291-0.0124j, -0.1243-0.0125j],
       [ 0.3416-0.1573j, -0.2791-0.148j , -0.2816-0.1608j, ...,
         0.5124+0.5065j,  0.2544+0.279j ,  0.2522+0.3207j],
       [-0.0447-0.0348j, -0.0473-0.0349j, -0.0483-0.0346j, ...,
        -0.1598-0.0051j, -0.1594-0.0067j, -0.1592-0.0112j],
       ...,
       [-0.133 -0.0436j, -0.1051-0.0644j, -0.078 -0.0867j, ...,
        -0.156 +0.1021j, -0.1438+0.0432j, -0.1106-0.006j ],
       [ 0.2182+0.1123j,  0.3067+0.0383j,  0.2961-0.0329j, ...,
        -0.0586+0.0723j, -0.0434+0.0244j, -0.0587+0.0083j],
       [ 0.1311+0.2634j,  0.1947+0.266j ,  0.1697+0.1318j, ...,
        -0.3065-0.0724j, -0.2364-0.0343j, -0.2099-0.0333j]])

In [10]:
# Drifter Dataset (u+iv,lat,qd,time)
ds = xr.Dataset({'uv_drifters': (['lat', 'time'], uv_drifters)},
                 coords={'lat': (['lat'], lat),
                         'lon': (['lat'], lon),
                         'time':(['time'], np.arange(1440)*3600),
                         'qd':(['lat'], qd)})
print(ds)
print('\n data size: %.1f GB' %(ds.nbytes / 1e9))

<xarray.Dataset>
Dimensions:      (lat: 126303, time: 1440)
Coordinates:
  * lat          (lat) float64 39.52 39.68 40.44 32.82 33.22 33.98 35.36 ...
    lon          (lat) float64 150.1 152.0 152.8 149.4 151.9 155.7 163.3 ...
  * time         (time) int64 0 3600 7200 10800 14400 18000 21600 25200 ...
    qd           (lat) uint8 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 ...
Data variables:
    uv_drifters  (lat, time) complex128 (-0.2105+0.2479j) (-0.1988+0.2534j) ...

 data size: 2.9 GB


# 2. FFT

In [11]:
#uv_fft_complex = xrft.dft(ds.uv_drifters.where(ds.qd>0).fillna(0.), dim=['time'], shift=False, detrend=False, window=False, chunks_to_segments=False)
uv_fft_complex = xrft.dft(ds.uv_drifters.fillna(0.), dim=['time'], shift=False, detrend='linear', window=True, chunks_to_segments=False)

E_uv_blocks = (uv_fft_complex*uv_fft_complex.conj()).where(uv_fft_complex!=0., drop=False)/1440*3600
E_uv_blocks = np.real(E_uv_blocks).rename('E_uv_blocks')
E_uv_blocks = E_uv_blocks.assign_coords(freq_cpd=E_uv_blocks.freq_time * 86400.) 
E_uv_blocks = E_uv_blocks.assign_coords(lon=ds.lon) 
print(E_uv_blocks)

<xarray.DataArray 'E_uv_blocks' (lat: 126303, freq_time: 1440)>
array([[5.168323e+03, 1.245437e+03, 1.023495e+04, ..., 4.618687e+04,
        3.516197e+04, 6.179483e+03],
       [2.378779e+03, 4.025700e+03, 4.582157e+03, ..., 1.510027e+04,
        3.714768e+04, 1.173497e+04],
       [1.206570e+04, 7.811274e+03, 9.585351e+02, ..., 3.269786e+04,
        9.335515e+04, 4.095906e+04],
       ...,
       [3.168327e+02, 2.288416e+02, 5.215848e+01, ..., 6.697984e+02,
        1.615073e+02, 5.425858e+02],
       [5.990080e+03, 1.716382e+04, 1.008467e+04, ..., 2.475921e+03,
        2.167523e+03, 5.568971e+03],
       [7.699786e+03, 1.762691e+03, 4.510642e+03, ..., 3.871838e+03,
        2.079355e+04, 2.952940e+04]])
Coordinates:
  * lat                (lat) float64 39.52 39.68 40.44 32.82 33.22 33.98 ...
  * freq_time          (freq_time) float64 0.0 1.929e-07 3.858e-07 5.787e-07 ...
    freq_time_spacing  float64 1.929e-07
    freq_cpd           (freq_time) float64 0.0 0.01667 0.03333 0.05 0.06667

In [12]:
df=E_uv_blocks.freq_time[1]-E_uv_blocks.freq_time[0]

In [13]:
f_drifter = 2.*2.*np.pi/86400. * np.sin(np.deg2rad(E_uv_blocks.lat))
f_cpd_drifter = f_drifter*86400/2./np.pi

In [16]:
E_all = (E_uv_blocks*df).sum(dim='freq_time')
E_all = np.real(E_all).rename('E_all')

E_semi = (E_uv_blocks.where((abs(E_uv_blocks.freq_cpd)>1.9) & (abs(E_uv_blocks.freq_cpd)<2.1) , other=0.)*df).sum(dim='freq_time')
E_semi = np.real(E_semi).rename('E_semi')

E_diurnal = (E_uv_blocks.where((abs(E_uv_blocks.freq_cpd)>0.9) & (abs(E_uv_blocks.freq_cpd)<1.1) , other=0.)*df).sum(dim='freq_time')
E_diurnal = np.real(E_diurnal).rename('E_diurnal')

E_low = (E_uv_blocks.where((abs(E_uv_blocks.freq_cpd)>=0) & (abs(E_uv_blocks.freq_cpd)<0.5) , other=0.)*df).sum(dim='freq_time')
E_low  = np.real(E_low).rename('E_low')

E_high = (E_uv_blocks.where((abs(E_uv_blocks.freq_cpd)>=0.5), other=0.)*df).sum(dim='freq_time')
E_high  = np.real(E_high).rename('E_high')

E_f = (E_uv_blocks.where((abs(E_uv_blocks.freq_cpd+f_cpd_drifter)<0.1), other=0.)*df).sum(dim='freq_time')
E_f = np.real(E_f).rename('E_f')

E_super = (E_uv_blocks.where((abs(E_uv_blocks.freq_cpd)>=2.1), other=0.)*df).sum(dim='freq_time')
E_super = np.real(E_super).rename('E_super')

In [19]:
E_drifter = xr.merge([E_all.to_dataset(), E_semi.to_dataset(), E_diurnal.to_dataset(), E_low.to_dataset(), E_high.to_dataset(), E_f.to_dataset(), E_super.to_dataset()])
file_E = data_dir+'mit_spec/E_drifter.nc'
%time E_drifter.to_netcdf(file_E, mode='w')  

CPU times: user 8 ms, sys: 4 ms, total: 12 ms
Wall time: 62.7 ms


/home1/datahome/xyu/.miniconda3/envs/equinox/lib/python3.6/site-packages/dask/utils.py:1010: UserWarning: Deprecated, see dask.base.get_scheduler instead
  warnings.warn("Deprecated, see dask.base.get_scheduler instead")


In [22]:
cluster.scheduler.close()

<Future finished result=None>